<a href="https://colab.research.google.com/github/kavyajeetbora/encode_polyline/blob/master/Encode_Polyline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title Encode GIS geometry to Encoded Polyline Algorithm Format
!pip install -q polyline
!pip install -q fiona

import polyline
import geopandas as gpd
import fiona
from ipywidgets import FileUpload
from IPython.display import display
from pathlib import Path
from tqdm.notebook import tqdm

## Functions for processing

def get_coords_from_linestring(linestring):
    y,x = linestring.xy
    x = list(x)
    y=list(y)
    coords = list(zip(x,y))
    return coords

def encode_geometry(gpkg_file):

    layers = fiona.listlayers(gpkg_file)
    for layer in tqdm(layers, unit=' layers'):
        gdf = gpd.read_file(gpkg_file, layer=layer)
        gdf = gdf.to_crs("EPSG:4326") ## Reproject to WGS84

        ## make sure all the linestrings are in type: shapely.geometry.LineString
        ## Explode the multipart geeomtry to single goemetry
        gdf = gdf.explode(index_parts=False).reset_index()

        ## Now get the list of coordinates from each LineString object
        coords = gdf['geometry'].apply(get_coords_from_linestring)

        ## Now encode all the coords to Encoded Polyline Algorithm Format
        gdf['encoded_polyline'] = coords.apply(lambda x: polyline.encode(x,precision=5))
        gdf = gdf.drop('geometry',axis=1)

        return gdf

# Create a file uploader widget
uploader = FileUpload(accept='.gpkg')

# Define a function to handle the uploaded file
def on_upload(change):

    ## get the filename
    uploaded_filename = next(iter(uploader.value))

    ## get the content of the file in binary
    content = uploader.value[uploaded_filename]['content']

    ## write the file to current directory
    with open(uploaded_filename, 'wb') as f:
        f.write(content)
    print(f"{uploaded_filename} has been uploaded successfully!")

    print("Encoding the geometry to Encoded Polyline Algorithm Format")
    ## Now encode the geometry
    gdf = encode_geometry(uploaded_filename)
    print("Successfully encoded the polyline geometries")

    ## Export it as csv
    filename = Path(uploaded_filename).stem
    gdf.to_csv(filename+".csv")
    print(f"Exported the data to {filename}.csv")

# Attach the function to the widget's observe method
uploader.observe(on_upload, names='_counter')

# Display the widget
display(uploader)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 78.5 MB/s eta 0:00:00


FileUpload(value={}, accept='.gpkg', description='Upload')